In [1]:
%pip install redis

import redis

r = redis.Redis(host='localhost', port=6379, db=0)


Note: you may need to restart the kernel to use updated packages.


In [2]:
from counter import get_last_counter

bank_id = f"bank_{get_last_counter()}"


In [3]:
r.keys()

[b'faiss_index:v1::bank_35',
 b'faiss_index:v1::bank_34',
 b'memory_banks:v1::bank_35',
 b'faiss_index:v1::bank_33',
 b'memory_banks:v1::bank_34',
 b'memory_banks:v1::bank_33',
 b'memory_banks:v1::bank_37',
 b'faiss_index:v1::bank_36',
 b'faiss_index:v1::bank_37',
 b'memory_banks:v1::bank_36']

In [4]:
r.get(f"memory_banks:v1::{bank_id}")

b'{"identifier":"bank_37","provider_resource_id":"bank_37","provider_id":"faiss","type":"memory_bank","memory_bank_type":"vector","embedding_model":"all-MiniLM-L6-v2","chunk_size_in_tokens":512,"overlap_size_in_tokens":null}'

In [6]:
r.info()

{'redis_version': '7.4.1',
 'redis_git_sha1': 0,
 'redis_git_dirty': 0,
 'redis_build_id': '7fd3520e9c14a41b',
 'redis_mode': 'standalone',
 'os': 'Linux 6.8.0-49-generic x86_64',
 'arch_bits': 64,
 'monotonic_clock': 'POSIX clock_gettime',
 'multiplexing_api': 'epoll',
 'atomicvar_api': 'c11-builtin',
 'gcc_version': '11.4.0',
 'process_id': 184895,
 'process_supervised': 'systemd',
 'run_id': 'a2cdc0a08c4d4eb4607a4792fb4e962e225ee8a6',
 'tcp_port': 6379,
 'server_time_usec': 1733637871975374,
 'uptime_in_seconds': 4488,
 'uptime_in_days': 0,
 'hz': 10,
 'configured_hz': 10,
 'lru_clock': 5584623,
 'executable': '/usr/bin/redis-server',
 'config_file': '/etc/redis/redis.conf',
 'io_threads_active': 0,
 'listener0': {'name': 'tcp', 'bind': '-::1', 'port': 6379},
 'connected_clients': 3,
 'cluster_connections': 0,
 'maxclients': 10000,
 'client_recent_max_input_buffer': 8,
 'client_recent_max_output_buffer': 0,
 'blocked_clients': 0,
 'tracking_clients': 0,
 'pubsub_clients': 0,
 'watch

In [9]:
from json import loads
from IPython.display import display

index = loads(r.get(f"faiss_index:v1::bank_33"))
display(index.keys())
display(len(index['id_by_index']))

dict_keys(['id_by_index', 'chunk_by_index', 'faiss_index'])

9923

In [ ]:
import os
from llama_stack_client import LlamaStackClient

# Test run
memory_bank_name = os.getenv("MEMORY_BANK_NAME")

client = LlamaStackClient(
    base_url="http://localhost:5001",
)

response = client.memory.query(
    bank_id=memory_bank_name,
    query=["Hello World"],
    params={"max_chunks": 10},
)

display(response)

for chunk, score in zip(response.chunks, response.scores):
    print(chunk)
    print(score)


In [ ]:
import pandas as pd

datasets = {
    "narrativeqa": "data/narrativeqa.parquet",
}

df = pd.read_parquet(datasets["narrativeqa"])
display(df.iloc[87]["questions"])
display(df.iloc[87]["context"])
df.head()